In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

import numpy as np
import arrayfire as af
from matplotlib import pyplot as plt

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import wave_equation_2d as w2d
from dg_maxwell import isoparam
from dg_maxwell import lagrange
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

[[ 0  9]
 [10 19]
 [20 29]
 [30 39]
 [40 49]
 [50 59]
 [60 69]
 [70 79]
 [80 89]
 [90 99]]
[[ 0 90]
 [ 1 91]
 [ 2 92]
 [ 3 93]
 [ 4 94]
 [ 5 95]
 [ 6 96]
 [ 7 97]
 [ 8 98]
 [ 9 99]]
gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


In [2]:
params.mesh_file = '../read_and_plot_mesh/mesh/square_10_10.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 441 nodes.
gmshTranslator: Mesh has 144 elements.
gmshTranslator: Processed 589 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


# Vectorise the u_at_edge function

In [3]:
# Write a vectorised edge retrieval function.

def u_at_edge(u_e_ij, edge_id):
    '''
    Finds the :math:`u` at given edge id for :math:`u_{eij}`.
    
    [Tested]
    [TODO] Documentation needed
    '''
    u_e_ij = af.moddims(u_e_ij, params.N_LGL, params.N_LGL)
    
    if edge_id == 0:
        return af.flat(u_e_ij[:, 0])
    
    elif edge_id == 3:
        return af.flat(u_e_ij[-1])
    
    elif edge_id == 2:
        return af.flat(u_e_ij[:, -1])
    
    elif edge_id == 1:
        return af.flat(u_e_ij[0])
    
    else:
        return
    
    return

# Get left edge of all the elements
u_e_ij = advec_var.u_e_ij

test_element_tag = 54
edge_id = 0

if edge_id == 0:

    u_e_ij = af.reorder(u_e_ij, d0 = 0, d1 = 2, d2 = 1)
    u_e_ij = af.moddims(u_e_ij, d0 = params.N_LGL, d1 = params.N_LGL, d2 = advec_var.elements.shape[0])

    u_edge = af.reorder(u_e_ij[:, 0, :], d0 = 0, d1 = 2, d2 = 1)
    print(u_edge.shape)
    print(af.all_true((u_edge[:, test_element_tag] - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)) == 0))


# Get bottom edge of all the elements

u_e_ij = advec_var.u_e_ij
edge_id = 1

if edge_id == 1:
    u_e_ij = advec_var.u_e_ij

    u_e_ij = af.reorder(u_e_ij, d0 = 0, d1 = 2, d2 = 1)
    u_e_ij = af.moddims(u_e_ij, d0 = params.N_LGL, d1 = params.N_LGL, d2 = advec_var.elements.shape[0])

    u_edge = af.reorder(u_e_ij[0, :, :], d0 = 1, d1 = 2, d2 = 0)
    print(u_edge.shape)
    print(af.all_true((u_edge[:, test_element_tag] - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)) == 0))


# Get right edge of all the elements
u_e_ij = advec_var.u_e_ij

edge_id = 2

if edge_id == 2:
    u_e_ij = advec_var.u_e_ij

    u_e_ij = af.reorder(u_e_ij, d0 = 0, d1 = 2, d2 = 1)
    u_e_ij = af.moddims(u_e_ij, d0 = params.N_LGL, d1 = params.N_LGL, d2 = advec_var.elements.shape[0])

    u_edge = af.reorder(u_e_ij[:, -1, :], d0 = 0, d1 = 2, d2 = 1)
    print(u_edge.shape)
    print(af.all_true((u_edge[:, test_element_tag] - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)) == 0))


# Get bottom edge of all the elements
u_e_ij = advec_var.u_e_ij

edge_id = 3

if edge_id == 3:
    u_e_ij = advec_var.u_e_ij

    u_e_ij = af.reorder(u_e_ij, d0 = 0, d1 = 2, d2 = 1)
    u_e_ij = af.moddims(u_e_ij, d0 = params.N_LGL, d1 = params.N_LGL, d2 = advec_var.elements.shape[0])

    u_edge = af.reorder(u_e_ij[-1, :, :], d0 = 1, d1 = 2, d2 = 0)
    print(u_edge.shape)
    print(af.all_true((u_edge[:, test_element_tag] - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)) == 0))


gmshTranslator: Ending
(8, 100)
1.0
(8, 100)
1.0
(8, 100)
1.0
(8, 100)
1.0


# Test

In [4]:
for test_element_tag in np.arange(100):

    edge_id = 0

    print(af.all_true((a2d_arbit_mesh.u_at_edge(advec_var.u_e_ij, edge_id, advec_var)[:, test_element_tag] \
          - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id) == 0)))

    edge_id = 1

    print(af.all_true((a2d_arbit_mesh.u_at_edge(advec_var.u_e_ij, edge_id, advec_var)[:, test_element_tag] \
          - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id) == 0)))

    edge_id = 2

    print(af.all_true((a2d_arbit_mesh.u_at_edge(advec_var.u_e_ij, edge_id, advec_var)[:, test_element_tag] \
          - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id) == 0)))

    edge_id = 3

    print(af.all_true((a2d_arbit_mesh.u_at_edge(advec_var.u_e_ij, edge_id, advec_var)[:, test_element_tag] \
          - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id) == 0)))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


# Get u_edge for a vector of element_tags

In [5]:
element_tag_vec = af.range(50, dtype = af.Dtype.u32)

# Get left edge of all the elements
u_e_ij = advec_var.u_e_ij

test_element_tag = 45
edge_id = 0

if edge_id == 0:

    u_e_ij = af.reorder(u_e_ij, d0 = 0, d1 = 2, d2 = 1)
    u_e_ij = af.moddims(u_e_ij, d0 = params.N_LGL, d1 = params.N_LGL, d2 = advec_var.elements.shape[0])

    u_edge = af.reorder(u_e_ij[:, 0, element_tag_vec], d0 = 0, d1 = 2, d2 = 1)
    print(u_edge.shape)
    print(af.all_true((u_edge[:, test_element_tag] - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)) == 0))


# Get bottom edge of all the elements

u_e_ij = advec_var.u_e_ij
edge_id = 1

if edge_id == 1:
    u_e_ij = advec_var.u_e_ij

    u_e_ij = af.reorder(u_e_ij, d0 = 0, d1 = 2, d2 = 1)
    u_e_ij = af.moddims(u_e_ij, d0 = params.N_LGL, d1 = params.N_LGL, d2 = advec_var.elements.shape[0])

    u_edge = af.reorder(u_e_ij[0, :, element_tag_vec], d0 = 1, d1 = 2, d2 = 0)
    print(u_edge.shape)
    print(af.all_true((u_edge[:, test_element_tag] - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)) == 0))


# Get right edge of all the elements
u_e_ij = advec_var.u_e_ij

edge_id = 2

if edge_id == 2:
    u_e_ij = advec_var.u_e_ij

    u_e_ij = af.reorder(u_e_ij, d0 = 0, d1 = 2, d2 = 1)
    u_e_ij = af.moddims(u_e_ij, d0 = params.N_LGL, d1 = params.N_LGL, d2 = advec_var.elements.shape[0])

    u_edge = af.reorder(u_e_ij[:, -1, element_tag_vec], d0 = 0, d1 = 2, d2 = 1)
    print(u_edge.shape)
    print(af.all_true((u_edge[:, test_element_tag] - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)) == 0))


# Get bottom edge of all the elements
u_e_ij = advec_var.u_e_ij

edge_id = 3

if edge_id == 3:
    u_e_ij = advec_var.u_e_ij

    u_e_ij = af.reorder(u_e_ij, d0 = 0, d1 = 2, d2 = 1)
    u_e_ij = af.moddims(u_e_ij, d0 = params.N_LGL, d1 = params.N_LGL, d2 = advec_var.elements.shape[0])

    u_edge = af.reorder(u_e_ij[-1, :, element_tag_vec], d0 = 1, d1 = 2, d2 = 0)
    print(u_edge.shape)
    print(af.all_true((u_edge[:, test_element_tag] - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)) == 0))


(8, 50)
1.0
(8, 50)
1.0
(8, 50)
1.0
(8, 50)
1.0


# Test

In [16]:
u_e_ij = advec_var.u_e_ij
element_tag_vec = af.range(50, dtype = af.Dtype.u32)


for test_element_tag in np.arange(50):

    edge_id = 0
    u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id)
    
    print(af.all_true((a2d_arbit_mesh.u_at_edge_element_wise(advec_var.u_e_ij, edge_id, element_tag_vec, advec_var)[:, test_element_tag] \
          - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id) == 0)))
    

    edge_id = 1

    print(af.all_true((a2d_arbit_mesh.u_at_edge_element_wise(advec_var.u_e_ij, edge_id, element_tag_vec, advec_var)[:, test_element_tag] \
          - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id) == 0)))

    edge_id = 2

    print(af.all_true((a2d_arbit_mesh.u_at_edge_element_wise(advec_var.u_e_ij, edge_id, element_tag_vec, advec_var)[:, test_element_tag] \
          - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id) == 0)))

    edge_id = 3

    print(af.all_true((a2d_arbit_mesh.u_at_edge_element_wise(advec_var.u_e_ij, edge_id, element_tag_vec, advec_var)[:, test_element_tag] \
          - u_at_edge(advec_var.u_e_ij[:, test_element_tag], edge_id) == 0)))

RuntimeError: In function af::dim4 getOutDims(const af::dim4&, const af::dim4&, bool)
In file src/backend/ArrayInfo.cpp:184
Invalid dimension for argument 1
Expected: ldims[i] == rdims[i] || ldims[i] == 1 || rdims[i] == 1


# Code to handle unshared edges

In [7]:
element_tags = af.range(advec_var.elements.shape[0], dtype = af.Dtype.u32)
value_at_the_shared_edge = 9.

# Left edge
edge_id = 0
interelement_relations = advec_var.interelement_relations
select_unshared_edges_0 = af.transpose(interelement_relations[:, edge_id] != -1)
select_unshared_edges_1 = af.transpose(interelement_relations[:, edge_id] == -1)

u_at_edge = a2d_arbit_mesh.u_at_edge_element_wise(advec_var.u_e_ij, edge_id, element_tags, advec_var)
u_at_edge = af.broadcast(utils.multiply, select_unshared_edges_0, u_at_edge)
u_at_edge = af.broadcast(utils.add, select_unshared_edges_1 * value_at_the_shared_edge,
                         u_at_edge)
print(af.transpose(u_at_edge))


RuntimeError: In function af::dim4 getOutDims(const af::dim4&, const af::dim4&, bool)
In file src/backend/ArrayInfo.cpp:178
Invalid dimension for argument 1
Expected: ldims == rdims


# Testing

In [9]:
u_edge = a2d_arbit_mesh.u_at_edge_element_wise(advec_var.u_e_ij, edge_id, element_tags, advec_var, unshared_edge_value = 9)
print(af.transpose(u_edge))

arrayfire.Array()
Type: double

[100 8 1 1]
    0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000     0.0000 
    0.0003     0.0003     0.0002     0.0001     0.0001     0.0001     0.0000     0.0000 
    0.0019     0.0017     0.0013     0.0008     0.0005     0.0003     0.0002     0.0002 
    0.0067     0.0059     0.0044     0.0029     0.0018     0.0012     0.0008     0.0007 
    0.0143     0.0125     0.0094     0.0062     0.0039     0.0025     0.0018     0.0015 
    0.0183     0.0161     0.0120     0.0080     0.0050     0.0032     0.0023     0.0019 
    0.0143     0.0125     0.0094     0.0062     0.0039     0.0025     0.0018     0.0015 
    0.0067     0.0059     0.0044     0.0029     0.0018     0.0012     0.0008     0.0007 
    0.0019     0.0017     0.0013     0.0008     0.0005     0.0003     0.0002     0.0002 
    0.0003     0.0003     0.0002     0.0001     0.0001     0.0001     0.0000     0.0000 
    0.0002     0.0002     0.0001     0.0001     0.0001     0.0001 

# Vectorize u_at_edge to take a vector of edge ID's

In [81]:
interelement_relations = advec_var.interelement_relations
left_edge_id = 0
left_edge_other_elements = interelement_relations[:, left_edge_id]

element_tags = af.range(advec_var.elements.shape[0], dtype = af.Dtype.u32)
other_element_edge_id = interelement_relations[left_edge_other_elements] != af.tile(element_tags, d0 = 1, d1 = 4)
other_element_edge_id = af.cast(af.iszero(other_element_edge_id), dtype = af.Dtype.u32)

# edge_ids = af.range(4, dtype= af.Dtype.u32)
# other_element_edge_id = af.broadcast(utils.multiply, af.transpose(edge_ids), other_element_edge_id)
# other_element_edge_id = af.sum(other_element_edge_id, dim = 1)

# [LOOKS FINE]

In [82]:
u_e_ij = advec_var.x_e_ij
element_tags = advec_var.interelement_relations[:, 0]

In [83]:
left_edge_id = 0
u_left = a2d_arbit_mesh.u_at_edge(u_e_ij,
                                  edge_id = left_edge_id,
                                  advec_var = advec_var)

# Bottom edge
bottom_edge_id = 1
u_bottom = a2d_arbit_mesh.u_at_edge(u_e_ij,
                                    edge_id = bottom_edge_id,
                                    advec_var = advec_var)

# Right edge
right_edge_id = 2
u_right = a2d_arbit_mesh.u_at_edge(u_e_ij,
                                   edge_id = right_edge_id,
                                   advec_var = advec_var)

# Top edge
top_edge_id = 3
u_top = a2d_arbit_mesh.u_at_edge(u_e_ij,
                                 edge_id = top_edge_id,
                                 advec_var = advec_var)


In [84]:
# Create a u_edge_vec

u_edge_vec = af.constant(0., d0 = params.N_LGL, d1 = advec_var.elements.shape[0], d2 = 4, dtype = af.Dtype.f64)
u_edge_vec[:, :, 0] = u_left
u_edge_vec[:, :, 1] = u_bottom
u_edge_vec[:, :, 2] = u_right
u_edge_vec[:, :, 3] = u_top

In [85]:
u_edge_other_element_vec = u_edge_vec[:, advec_var.interelement_relations[:, 0], :]
# [SEEMS FINE]

In [138]:
u_edge_other_element = af.broadcast(utils.multiply,
                                    af.reorder(other_element_edge_id,
                                               d0 = 2, d1 = 0, d2 = 1),
                                    u_edge_other_element_vec)

# print(af.all_true(af.abs(af.transpose(u_edge_other_element[:, :, 1])) < 1e-14))
# # [SEEMS FINE]

u_edge_other_element = af.sum(u_edge_other_element, dim = 2)
# print(af.transpose(u_edge_other_element))
# [SEEMS FINE]

# Writing formal code for modifing u_at_edge_elementwise for giving u for other element edges

In [40]:
# Arguments

u_e_ij = advec_var.y_e_ij
element_edge_id = 0

other_element_tags = advec_var.interelement_relations[:, element_edge_id]

# Create a u_edge_vec
# Left edge
left_edge_id = 0
u_left = a2d_arbit_mesh.u_at_edge(u_e_ij,
                                  edge_id = left_edge_id,
                                  advec_var = advec_var)
# Bottom edge
bottom_edge_id = 1
u_bottom = a2d_arbit_mesh.u_at_edge(u_e_ij,
                                    edge_id = bottom_edge_id,
                                    advec_var = advec_var)
# Right edge
right_edge_id = 2
u_right = a2d_arbit_mesh.u_at_edge(u_e_ij,
                                   edge_id = right_edge_id,
                                   advec_var = advec_var)
# Top edge
top_edge_id = 3
u_top = a2d_arbit_mesh.u_at_edge(u_e_ij,
                                 edge_id = top_edge_id,
                                 advec_var = advec_var)

u_edge_vec = af.constant(0., d0 = params.N_LGL, d1 = advec_var.elements.shape[0], d2 = 4, dtype = af.Dtype.f64)
u_edge_vec[:, :, 0] = u_left
u_edge_vec[:, :, 1] = u_bottom
u_edge_vec[:, :, 2] = u_right
u_edge_vec[:, :, 3] = u_top

In [41]:
element_tags = af.range(advec_var.elements.shape[0], dtype = af.Dtype.u32)
other_element_edge_id = advec_var.interelement_relations[other_element_tags] != af.tile(element_tags, d0 = 1, d1 = 4)
other_element_edge_id = af.cast(af.iszero(other_element_edge_id), dtype = af.Dtype.u32)

u_edge_other_element_vec = u_edge_vec[:, advec_var.interelement_relations[:, element_edge_id], :]

u_edge_other_element = af.broadcast(utils.multiply,
                                    af.reorder(other_element_edge_id,
                                               d0 = 2, d1 = 0, d2 = 1),
                                    u_edge_other_element_vec)

u_edge_other_element = af.sum(u_edge_other_element, dim = 2)
# print(af.transpose(u_edge_other_element))

In [42]:
ref_u_at_edge_element_wise = a2d_arbit_mesh.u_at_edge_element_wise(u_e_ij, 2, advec_var.interelement_relations[:, element_edge_id], advec_var)
print(af.all_true(af.abs(u_edge_other_element - ref_u_at_edge_element_wise) < 1e-14))
# print(af.transpose(u_edge_other_element))
# print(af.transpose(ref_u_at_edge_element_wise))
# print(advec_var.interelement_relations[:, element_edge_id])

1.0


In [43]:
u_at_edge_other_element = a2d_arbit_mesh.u_at_other_element_edge(u_edge_vec = u_edge_vec,
                                                                 element_edge_id = element_edge_id,
                                                                 advec_var = advec_var)
print(af.all_true(af.abs(u_edge_other_element - ref_u_at_edge_element_wise) < 1e-14))

1.0
